In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [130]:
import pandas as pd
from tqdm.notebook import tqdm
import os
from shutil import copyfile, move, copy
import sys
import json

In [20]:
!pip install gitpython

In [105]:
!rm -r /kaggle/working/

rm: cannot remove '/kaggle/working/': Device or resource busy


In [106]:
import git
git.Repo.clone_from('https://github.com/ultralytics/JSON2YOLO.git', '/kaggle/working/JSON2YOLO')

<git.repo.base.Repo '/kaggle/working/JSON2YOLO/.git'>

In [107]:
!ls /kaggle/working/JSON2YOLO/

LICENSE    general_json2yolo.py   requirements.txt
README.md  labelbox_json2yolo.py  utils.py


In [108]:
with open('/kaggle/working/JSON2YOLO/general_json2yolo.py', 'r+') as fd:
    lines = fd.readlines()                                                     
    fd.seek(0)                                                                 
    line273 = ["            h, w, f = img['height'], img['width'], img['file_name'].split('/')[1]"]
    print(line273)
    fd.writelines(lines[:273] + line273 + lines[274:])

["            h, w, f = img['height'], img['width'], img['file_name'].split('/')[1]"]


In [109]:
fd = open('/kaggle/working/JSON2YOLO/general_json2yolo.py', 'r')                                             
lines = fd.readlines()                                                     
fd.seek(0)                                                                 
print(lines[273])
fd.close()

            h, w, f = img['height'], img['width'], img['file_name'].split('/')[1]



In [131]:
sys.path.append('/kaggle/input/rtsd-dataset')
sys.path.append('/kaggle/working/JSON2YOLO')
from JSON2YOLO.general_json2yolo import convert_coco_json

test_path = '/kaggle/working/test_annotation/'
train_path = '/kaggle/working/train_annotation/'

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

copy('/kaggle/input/rtsd-dataset/train_anno.json', os.path.join(train_path, 'train_anno.json'))
copy('/kaggle/input/rtsd-dataset/val_anno.json', os.path.join(test_path, 'val_anno.json'))

for folder in ['labels', 'images']:
    for path in [test_path, train_path]:
        os.makedirs(os.path.join(path, folder), exist_ok=True)
        
convert_coco_json(train_path)
for file in tqdm(os.listdir(os.path.join('new_dir/labels/train_anno'))):
    move(os.path.join('new_dir/labels/train_anno', file), os.path.join(train_path, 'labels', file))
    
convert_coco_json('./test_annotation/')
for file in tqdm(os.listdir(os.path.join('new_dir/labels/val_anno'))):
    move(os.path.join('new_dir/labels/val_anno', file), os.path.join(test_path, 'labels', file))

Annotations /kaggle/working/train_annotation/train_anno.json: 100%|██████████| 54188/54188 [00:06<00:00, 7774.84it/s]


  0%|          | 0/54188 [00:00<?, ?it/s]

Annotations /kaggle/working/test_annotation/val_anno.json: 100%|██████████| 5000/5000 [00:00<00:00, 7757.33it/s]


  0%|          | 0/5000 [00:00<?, ?it/s]

In [137]:
test_labels = os.listdir(os.path.join(test_path, 'labels'))
train_labels = os.listdir(os.path.join(train_path, 'labels'))

test_labels = set(map(lambda x: x.split('.')[0], test_labels))
train_labels = set(map(lambda x: x.split('.')[0], train_labels))

images = '/kaggle/input/rtsd-dataset/rtsd-frames/rtsd-frames'
for file in os.listdir(images):
    name = file.split('.')[0]
    if name in train_labels:
        copy(os.path.join(images, file), os.path.join(train_path,'images', file))
    if name in test_labels:
        copy(os.path.join(images, file), os.path.join(test_path,'images', file))

In [ ]:
!pip install ultralytics

In [1]:
from ultralytics import YOLO

In [141]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 131MB/s]


In [ ]:
data_yaml = '/kaggle/input/trafic-signs-yaml/trafic_signs.yaml'
results = model.train(data=data_yaml, epochs=100,batch=1, imgsz=1280)

Ultralytics YOLOv8.0.209 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/trafic-signs-yaml/trafic_signs.yaml, epochs=100, patience=50, batch=1, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
train: Scanning /kaggle/working/train_annotation/labels... 54188 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54188/54188 [00:45<00:00, 1183.71it/s]
train: New cache created: /kaggle/working/train_annotation/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /kaggle/working/test_annotation/labels... 5000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:04<00:00, 1169.75it/s]
val: New cache created: /kaggle/working/test_annotation/labels.cache
Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 

In [2]:
model = YOLO('last.pt')

In [20]:
import cv2

results = model(source='https://www.youtube.com/watch?v=DMXmKg6thHo', save=True)

#results = model.track(source='https://www.youtube.com/watch?v=DMXmKg6thHo', show=True, imgsz=(640,480))
#annotated_frame = results[0].plot()
#cv2.imshow("YOLOv8 Tracking", annotated_frame)
#if cv2.waitKey(1) & 0xFF == ord("q"):
#    break
#cv2.destroyAllWindows()


1/1: https://www.youtube.com/watch?v=DMXmKg6thHo... Success  (8598 frames of shape 1920x1080 at 29.97 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 736x1280 1 4_2_1, 173.0ms
0: 736x1280 (no detections), 376.0ms
0: 736x1280 (no detections), 196.0ms
0: 736x1280 (no detections), 261.0ms
0: 736x1280 (no detections), 235.0ms
0: 736x1280 (no detections), 200.0ms
0: 736x1280 (no detections), 99.0ms
0: 736x1280 (no detections), 54.0ms
0: 736x1280 (no detections), 236.0ms
0: 736x1280 (no detections)

0: 736x1280 (no detections), 57.0ms
0: 736x1280 (no detections), 59.0ms
0: 736x1280 (no detections), 107.0ms
0: 736x1280 (no detections), 153.0ms
0: 736x1280 (no detections), 100.0ms
0: 736x1280 (no detections), 54.0ms
0: 736x1280 (no detections), 181.0ms
0: 736x1280 (no detections), 108.0ms
0: 736x1280 (no detections), 69.0ms
0: 736x1280 (no detections), 77.0ms
0: 736x1280 (no detections), 307.5ms
0: 736x1280 1 7_3, 138.0ms
0: 736x1280 (no detections), 161.0ms
0: 736x1280 (no detections), 120.0ms
0: 736x1280 (no detections), 93.0ms
0: 736x1280 (no detections), 192.0ms
0: 736x1280 (no detections), 126.0ms
0: 736x1280 (no detections), 154.0ms
0: 736x1280 (no detections), 243.0ms
0: 736x1280 (no detections), 47.0ms
0: 736x1280 (no detections), 64.0ms
0: 736x1280 (no detections), 171.0ms
0: 736x1280 (no detections), 119.0ms
0: 736x1280 (no detections), 90.0ms
0: 736x1280 (no detections), 203.0ms
0: 736x1280 (no detections), 51.0ms
0: 736x1280 (no detections), 81.0ms
0: 736x1280 (no detect

0: 736x1280 1 2_1, 171.0ms
0: 736x1280 (no detections), 126.0ms
0: 736x1280 (no detections), 200.0ms
0: 736x1280 (no detections), 125.0ms
0: 736x1280 (no detections), 118.0ms
0: 736x1280 1 3_27, 112.0ms
0: 736x1280 (no detections), 54.0ms
0: 736x1280 (no detections), 97.0ms
0: 736x1280 (no detections), 52.0ms
0: 736x1280 (no detections), 194.0ms
0: 736x1280 1 2_4, 199.0ms
0: 736x1280 (no detections), 249.0ms
0: 736x1280 (no detections), 129.0ms
0: 736x1280 1 6_4, 91.0ms
0: 736x1280 (no detections), 267.0ms
0: 736x1280 (no detections), 146.0ms
0: 736x1280 (no detections), 186.0ms
0: 736x1280 (no detections), 164.0ms
0: 736x1280 (no detections), 154.0ms
0: 736x1280 (no detections), 219.0ms
0: 736x1280 (no detections), 63.0ms
0: 736x1280 (no detections), 117.0ms
0: 736x1280 (no detections), 240.0ms
0: 736x1280 (no detections), 213.0ms
0: 736x1280 (no detections), 238.0ms
0: 736x1280 (no detections), 196.0ms
0: 736x1280 (no detections), 69.0ms
0: 736x1280 (no detections), 272.0ms
0: 736x12

0: 736x1280 (no detections), 226.0ms
0: 736x1280 (no detections), 227.0ms
0: 736x1280 1 2_1, 33.0ms
0: 736x1280 (no detections), 84.0ms
0: 736x1280 (no detections), 63.0ms
0: 736x1280 (no detections), 75.0ms
0: 736x1280 (no detections), 241.0ms
0: 736x1280 (no detections), 160.0ms
0: 736x1280 (no detections), 188.0ms
0: 736x1280 (no detections), 199.0ms
0: 736x1280 1 2_1, 2 5_19_1s, 1 4_1_1, 108.0ms
0: 736x1280 1 2_1, 1 3_24, 170.0ms
0: 736x1280 (no detections), 174.0ms
0: 736x1280 (no detections), 75.0ms
0: 736x1280 (no detections), 50.0ms
0: 736x1280 1 1_23, 31.0ms
0: 736x1280 (no detections), 52.0ms
0: 736x1280 (no detections), 90.0ms
0: 736x1280 (no detections), 34.0ms
0: 736x1280 (no detections), 30.0ms
0: 736x1280 (no detections), 66.0ms
0: 736x1280 (no detections), 36.0ms
0: 736x1280 (no detections), 115.0ms
WARNING  Video stream unresponsive, please check your IP camera connection.
0: 736x1280 (no detections), 80.0ms
0: 736x1280 (no detections), 64.0ms
0: 736x1280 (no detection

In [40]:
cv2.destroyAllWindows()

In [25]:
annotated_frame = results[0].plot()
#cv2.imshow("YOLOv8 Tracking", annotated_frame)

In [29]:
from PIL import Image

i=0
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    fname = 'res_' + str(i) + '.jpg'
    im.save(fname)  # save image
    i+=1

WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Vid

In [39]:
for i, r in enumerate(results):
    if len(r.boxes.cls)!=0:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        fname = 'res_' + str(i) + '.jpg'
        im.save(fname)  # save image
        

WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.


# convert

In [41]:
model.export(format='onnx', half=True)

Ultralytics YOLOv8.0.209  Python-3.9.13 torch-2.1.0+cu121 CPU (AMD Phenom(tm) II X6 1055T Processor)
WARNING  half=True only compatible with GPU export, i.e. use device=0

PyTorch: starting from 'last.pt' with input shape (1, 3, 1280, 1280) BCHW and output shape(s) (1, 159, 33600) (26.1 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
  Obtaining dependency information for onnx>=1.12.0 from https://files.pythonhosted.org/packages/b8/d9/39382887c8fb4330a8372739a9534b6734c18012f18726f024d9cae40e96/onnx-1.15.0-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 14.3/14.3 MB 11.9 MB/s eta 0:00:00

requirements: AutoUpdate success  42.4s, installed 1 package: ['onnx>=1.12.0']
requirements:  Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success  51.3s, saved as 'last.onnx' (13.5 MB)

Export complete (66.9s)
Results saved to D:\\1 \   
Predict:

'last.onnx'

In [3]:
model.export(format='ncnn')

Ultralytics YOLOv8.0.209  Python-3.9.13 torch-2.1.0+cu121 CPU (AMD Phenom(tm) II X6 1055T Processor)
Model summary (fused): 168 layers, 3357389 parameters, 0 gradients, 9.7 GFLOPs

PyTorch: starting from 'last.pt' with input shape (1, 3, 1280, 1280) BCHW and output shape(s) (1, 159, 33600) (26.1 MB)

TorchScript: starting export with torch 2.1.0+cu121...
TorchScript: export success  50.2s, saved as 'last.torchscript' (13.5 MB)

ncnn: starting export with ncnn 1.0.20231027...
ncnn: running 'C:\anaconda3\Lib\site-packages\ultralytics\pnnx.exe last.torchscript ncnnparam=last_ncnn_model\model.ncnn.param ncnnbin=last_ncnn_model\model.ncnn.bin ncnnpy=last_ncnn_model\model_ncnn.py pnnxparam=last_ncnn_model\model.pnnx.param pnnxbin=last_ncnn_model\model.pnnx.bin pnnxpy=last_ncnn_model\model_pnnx.py pnnxonnx=last_ncnn_model\model.pnnx.onnx fp16=0 device=cpu inputshape="[1, 3, 1280, 1280]"'
ncnn: export success  9.5s, saved as 'last_ncnn_model' (13.5 MB)

Export complete (79.2s)
Results saved to

'last_ncnn_model'

In [ ]:
model.export(format='tflite')

Ultralytics YOLOv8.0.209  Python-3.9.13 torch-2.1.0+cu121 CPU (AMD Phenom(tm) II X6 1055T Processor)
Model summary (fused): 168 layers, 3357389 parameters, 0 gradients, 9.7 GFLOPs

PyTorch: starting from 'last.pt' with input shape (1, 3, 1280, 1280) BCHW and output shape(s) (1, 159, 33600) (26.1 MB)

TensorFlow SavedModel: starting export with tensorflow 2.10.1...

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success  29.3s, saved as 'last.onnx' (13.3 MB)
TensorFlow SavedModel: running 'onnx2tf -i "last.onnx" -o "last_saved_model" -nuo --non_verbose'


In [10]:
model.export(format='torchscript', optimize=True)

AttributeError: 'AutoShape' object has no attribute 'export'

In [10]:
import torch

from torch.utils.mobile_optimizer import optimize_for_mobile

torchscript_model = "last.torchscript"
export_model_name = "last.torchscript.ptl"

model = torch.jit.load(torchscript_model)
optimized_model = optimize_for_mobile(model)
optimized_model._save_for_lite_interpreter(export_model_name)

print(f"mobile optimized model exported to {export_model_name}")

mobile optimized model exported to last.torchscript.ptl


In [12]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\basil/.cache\torch\hub\ultralytics_yolov5_master
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
  Obtaining dependency information for gitpython>=3.1.30 from https://files.pythonhosted.org/packages/8d/c4/82b858fb6483dfb5e338123c154d19c043305b01726a67d89532b8f8f01b/GitPython-3.1.40-py3-none-any.whl.metadata
  Obtaining dependency information for gitdb<5,>=4.0.1 from https://files.pythonhosted.org/packages/fd/5b/8f0c4a5bb9fd491c277c21eff7ccae71b47d43c4446c9d0c6cff2fe8c2c4/gitdb-4.0.11-py3-none-any.whl.metadata
   ---------------------------------------- 190.6/190.6 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 62.7/62.7 kB ? eta 0:00:00

requirements: AutoUpdate success  13.3s, installed 1 package: ['gitpython>=3.1.30']
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-11-17 Python-3.9.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)



In [14]:
import git
git.Repo.clone_from('https://github.com/ultralytics/yolov5', '/git/yolo_v5')

<git.repo.base.Repo 'C:\\git\\yolo_v5\\.git'>

In [20]:
!python c:\\git\\yolo_v5\\export.py --weights best_y5.pt --include torchscript

export: data=C:\git\yolo_v5\data\coco128.yaml, weights=['best_y5.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript']
YOLOv5  v7.0-241-gb6a65e1 Python-3.9.13 torch-2.1.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7428160 parameters, 0 gradients, 17.1 GFLOPs

PyTorch: starting from best_y5.pt with output shape (1, 25200, 160) (14.5 MB)

TorchScript: starting export with torch 2.1.0+cu121...
TorchScript: export success  16.1s, saved as best_y5.torchscript (28.8 MB)

Export complete (22.1s)
Results saved to D:\\1 \   
Detect:          python detect.py --weights best_y5.torchscript 
Validate:        python val.py --weights best_y5.torchscript 
PyTorch Hub:     model = torch.hub.load('ultralytics/yolov5', 'custom', 'best_y5.to

In [21]:
import torch

from torch.utils.mobile_optimizer import optimize_for_mobile

torchscript_model = "best_y5.torchscript"
export_model_name = "best_y5.torchscript.ptl"

model = torch.jit.load(torchscript_model)
optimized_model = optimize_for_mobile(model)
optimized_model._save_for_lite_interpreter(export_model_name)

print(f"mobile optimized model exported to {export_model_name}")

mobile optimized model exported to best_y5.torchscript.ptl
